In [ ]:
import pandas as pd

# Load the dataframe.
df = pd.read_csv('/content/grid_asset_data.csv')

In [ ]:
# Clean Dataset
df['Fault_Event'] = df['Fault_Event'].fillna('No fault')
df = df.drop(columns=['Timestamp', 'Substation_ID', 'Asset_ID', 'Reconfig_Action', 'Asset_Type', 'Load_Type'], errors='ignore')
display(df.head())

,Voltage_V,Current_A,Power_kW,Frequency_Hz,Energy_Consumed_kWh,Fault_Event
0,232.48,9.59,2.23,50.13,0.0372,No fault
1,237.62,9.30,2.21,49.95,0.0368,No fault
2,237.90,12.30,2.93,49.91,0.0488,No fault
3,232.71,8.61,2.00,49.91,0.0333,No fault
4,231.21,4.26,0.98,49.66,0.0163,No fault


In [ ]:
data_for_synthesis = df[['Energy_Consumed_kWh', 'Voltage_V', 'Current_A', 'Power_kW', 'Frequency_Hz']].copy()
display(data_for_synthesis.head())
display(data_for_synthesis.info())

,Energy_Consumed_kWh,Voltage_V,Current_A,Power_kW,Frequency_Hz
0,0.0372,232.48,9.59,2.23,50.13
1,0.0368,237.62,9.30,2.21,49.95
2,0.0488,237.90,12.30,2.93,49.91
3,0.0333,232.71,8.61,2.00,49.91
4,0.0163,231.21,4.26,0.98,49.66


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440 entries, 0 to 1439
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Energy_Consumed_kWh  1440 non-null   float64
 1   Voltage_V            1440 non-null   float64
 2   Current_A            1440 non-null   float64
 3   Power_kW             1440 non-null   float64
 4   Frequency_Hz         1440 non-null   float64
dtypes: float64(5)
memory usage: 56.4 KB


None

In [ ]:
!pip install sdv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.6/185.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.8/73.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/

In [ ]:
!pip install sdv[tabular]

In [ ]:
from sdv.single_table import TVAESynthesizer
from sdv.metadata import SingleTableMetadata
from sdv.sampling import Condition
import pandas as pd

# Define metadata (especially data types)
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=df)

# Train model
model = TVAESynthesizer(metadata)
model.fit(df)

/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:167: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:133: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


In [ ]:
# Test
# Conditional sampling - create proper Condition objects
conditions = [
    Condition(
        column_values={'Energy_Consumed_kWh': 0.05},
        num_rows=1  # Specify how many rows you want to generate
    )
]

# Generate synthetic data with conditions
synthetic_data = model.sample_from_conditions(conditions=conditions)

# Display the head of the generated synthetic data DataFrame
display(synthetic_data.head())

# Display the info of the generated synthetic data DataFrame
display(synthetic_data.info())

Sampling conditions: 100%|██████████| 1/1 [00:09<00:00,  9.85s/it]


,Voltage_V,Current_A,Power_kW,Frequency_Hz,Energy_Consumed_kWh,Fault_Event
0,237.24,12.72,2.5,50.12,0.05,No fault


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Voltage_V            1 non-null      float64
 1   Current_A            1 non-null      float64
 2   Power_kW             1 non-null      float64
 3   Frequency_Hz         1 non-null      float64
 4   Energy_Consumed_kWh  1 non-null      float64
 5   Fault_Event          1 non-null      object 
dtypes: float64(5), object(1)
memory usage: 180.0+ bytes


None

In [ ]:
#Using QualityReport to provide overall score synthetic data quality

# Try alternative import paths
try:
    from sdmetrics.reports.single_table import QualityReport
    print("Found QualityReport in sdmetrics.reports.single_table")

    # Generate synthetic data
    synthetic_data_eval = loaded_model.sample(num_rows=len(df))

    # Create and generate report
    report = QualityReport()
    report.generate(df, synthetic_data_eval, metadata.to_dict())

    print("Overall Quality Score:", report.get_score())

except ImportError:
    print("QualityReport not found in sdmetrics.reports.single_table either")

Found QualityReport in sdmetrics.reports.single_table
Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 6/6 [00:00<00:00, 451.48it/s]|
Column Shapes Score: 87.16%

(2/2) Evaluating Column Pair Trends: |██████████| 15/15 [00:00<00:00, 165.18it/s]|
Column Pair Trends Score: 87.77%

Overall Score (Average): 87.47%

Overall Quality Score: 0.8746533425148919


In [ ]:
# Save the trained model
model.save('ctgan_grid_asset_model.pkl')